In [ ]:
from selenium import webdriver    # to  control the chrome browser
from selenium.webdriver.common.by import By
import time
from datetime import datetime, date
import bs4
from bs4 import BeautifulSoup     # to parse the page source
import pandas as pd                # to create csv file of scraped user details
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import random
import re
from pathlib import Path
from requests.exceptions import RequestException

In [ ]:
url = 'https://www.fredhutch.org/en/faculty-lab-directory.html'
base_url = 'https://www.fredhutch.org'

In [ ]:
def create_driver(url):
    chrome_options = Options()
    chrome_options.headless=True
    driver = webdriver.Chrome('/Volumes/Users/matu/Documents/Xcode/chromedriver',options=chrome_options )
    driver.get(url)
    return driver

def scrap_cards(block):
    data = {}
    try:
        data['name'] = block.h3.text
        data['url'] = f"{base_url}{block.div.h4.a.get('href')}"
        ps = block.find('p', attrs={'class':'appt1'})
        data['title'] = ps.span.text
        data['department'] =ps.contents[-1].strip(' ,').split(',')[0]
    except: print(block)
    finally:
        return data
    
def load_more(driver):
    try:
        more = driver.find_element_by_class_name('more')
        more.click()
        time.sleep(random.randint(7,15))
        return True
    except:
        return False

In [ ]:
driver = create_driver('https://www.fredhutch.org/en/faculty-lab-directory/beresford-shirley.html')
soup = BeautifulSoup(driver.page_source, 'html5')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


In [ ]:
scrap_profile(soup)

{'phone': '206.667.4793',
 'email': 'sberesfo@fredhutch.org',
 'fax': '206.667.5977',
 'mail stop': 'm3-b232',
 'summary': 'UW Contact Info\nPhone: 206.543.9512\nFax: 206.695.9651\nberesfrd@u.washington.edu\n',
 'job_description': 'Nutritional epidemiology and chronic disease prevention, specifically designed to improve the scientific basis for public health policy and recommendations concerning dietary intake, physical activity and intake of folic acid.'}

In [ ]:
def get_pis(driver):
    data = []
    while load_more(driver):
        continue
    soup = BeautifulSoup(driver.page_source, 'html5')
    blocks = soup.findAll('div', attrs={'class':'cmp-faculty-and-labs__card'})
    for block in blocks:
        card = scrap_cards(block)
        data.append(card)
    return data

def get_profiles(driver, pi):
    # for pi in data:
    driver.get(pi['url'])
    time.sleep(random.randint(7,10))
    soup = BeautifulSoup(driver.page_source, 'html5')
    profile = scrap_profile(soup)
    pi.update(profile)
    return pi

def scrap_profile(soup):
    data = {}
    # driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html5')
    for contact in soup.findAll('div', attrs={'class':'facultyContactInfo__item'}):
        key = contact.find('span', attrs={'facultyContactInfo__label'}).text.lower().strip(' :')
        data[key] = contact.find('span', attrs={'facultyContactInfo__value'}).text.lower().strip(' :\n\t\r')
    texts = soup.findAll('div',attrs={'class':'cmp-text'})
    text_blocks = [x for x in texts if not x.div]
    data['summary'] = text_blocks[0].p.text if text_blocks[0].p else ''
    #Getting research info
    research = soup.find('div',attrs={'class':'research-wrapper'})
    if research:
        data['pos_description'] = research.p.text
    else:
        research_blocks = [x for x in texts if not x.div and x.h3 and x.h3.text.find('Research')>=0]
        if len(research_blocks) > 0:
            data['pos_description'] =  " ".join([x.text for x in research_blocks[0].findAll('p')])
    return data        


In [ ]:
driver = create_driver(url)
pis = get_pis(driver)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


In [ ]:
pi_org = pis.copy()

In [ ]:
profiles = [get_profiles(driver, pi) for pi in pis]

In [ ]:
df = pd.DataFrame(profiles)

In [ ]:
df.head()

,name,url,title,department,phone,email,mail stop,summary,job_description,fax,office
0,"Jennifer E. Adair, Ph.D.",https://www.fredhutch.org/en/faculty-lab-direc...,Associate Professor,Clinical Research Division,206.667.7110,jadair@fredhutch.org,s3-204,Dr. Jennifer Adair is developing gene therapie...,Gene therapy in hematopoietic stem cells has e...,NaN,NaN
1,"Richard Adeyemi, D.V.M., Ph.D.",https://www.fredhutch.org/en/faculty-lab-direc...,Assistant Professor,Basic Sciences Division,206.667.6263,radeyemi@fredhutch.org,a1-162,Dr. Richard Adeyemi studies how our cells main...,NaN,NaN,NaN
2,"Naveed Ali, M.D.",https://www.fredhutch.org/en/faculty-lab-direc...,Assistant Professor,Clinical Research Division,206.667.5854,nali2@fredhutch.org,d5-310,Dr. Naveed Ali studies new treatment regimens ...,NaN,NaN,NaN
3,"James V. Alvarez, Ph.D.",https://www.fredhutch.org/en/faculty-lab-direc...,Associate Professor,Public Health Sciences Division,206.667.6348,jalvarez@fredhutch.org,m5-c800,Cancer cell biologist Dr. James Alvarez studie...,NaN,NaN,NaN
4,"Benjamin O. Anderson, M.D.",https://www.fredhutch.org/en/faculty-lab-direc...,"Professor, Epidemiology",Public Health Sciences Division,206.543.6352,banderso@fredhutch.org,m4-a402,Dr. Ben Anderson is an internationally recogni...,NaN,206.543.8136,NaN


In [ ]:
df.to_csv('/Volumes/Users/matu/Downloads/LB_FH_webpage.csv', index=False)

In [ ]:
def search(query, df):
    df = df.copy()
    df['lname'] = df.name.apply(lambda x: x.lower())
    return df.loc[df.lname.str.contains(query.lower())]

In [ ]:
search('kristina', df)

,name,url,title,department,phone,email,mail stop,summary,job_description,fax,office,lname
